# modelo 02 - usar features extra

- https://chatgpt.com/share/67d9fe94-c20c-8009-81a3-63cf5310785f

In [ ]:
import pandas as pd

df_train = pd.read_pickle('Model_Train.pkl')
df_val = pd.read_pickle('Model_Val.pkl')

In [ ]:
set(df_train["img_placement"]) | set(df_val["img_placement"])

In [ ]:
def hot_encod_img_placement(df):
    df["img_placement_roof"] = df["img_placement"].apply(lambda x: 1 if x == "roof" else 0)
    df["img_placement_r_openspace"] = df["img_placement"].apply(lambda x: 1 if x == "r_openspace" else 0)
    df["img_placement_openspace"] = df["img_placement"].apply(lambda x: 1 if x == "openspace" else 0)
    df["img_placement_S-unknown"] = df["img_placement"].apply(lambda x: 1 if x == "S-unknown" else 0)
    df.drop(columns=["img_placement"], inplace=True)

hot_encod_img_placement(df_train)
hot_encod_img_placement(df_val)

In [ ]:
set(df_train["img_origin"]) | set(df_val["img_origin"])

In [ ]:
def hot_encod_img_placement(df):
    df["img_origin_D"] = df["img_origin"].apply(lambda x: 1 if x == "D" else 0)
    df["img_origin_S"] = df["img_origin"].apply(lambda x: 1 if x == "S" else 0)
    df.drop(columns=["img_origin"], inplace=True)

hot_encod_img_placement(df_train)
hot_encod_img_placement(df_val)

In [ ]:
df_train.sample(3)

In [ ]:
df_val.sample(3)

## modelo itself

ideia: modelo que com a mask + rbg consiga determinar a quantidade de boiler e pans

é preciso criar os dados primeiro

In [ ]:
from PIL import Image
from torchvision import transforms as T
import numpy as np
import cv2
import albumentations as A


transforms = T.Compose([
    T.Resize((512,512)),
    T.ToTensor(),
])

augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomRotate90(p=0.3),
    A.RandomBrightnessContrast(p=0.2),
    A.GaussianBlur(blur_limit=(3, 7), p=0.3),
    A.HueSaturationValue(p=0.3),
], additional_targets={'mask': 'mask'})

def polygons_to_segmentation_mask(polygons):
    image = np.zeros((512, 512), dtype=np.uint8)
    for polygon in polygons:
        poly = np.array([polygon], dtype=np.int32)
        cv2.fillPoly(image, [poly], 1)
    return image

def create_data(img_id, polygon, data=None):
    # 512 x 512 x 3
    image = Image.open(f"images/{img_id}.jpg").convert("RGB")
    image = transforms(image).permute(1, 2, 0).numpy()

    # 512 x 512
    mask = polygons_to_segmentation_mask(polygon).astype(int)

    # Apply augmentations
    if data == "train":
        augmented = augment(image=image, mask=mask)
        image_aug, mask_aug = augmented["image"], augmented["mask"]

    # 3 x 512 x 512
    image_masked = image_aug * np.repeat(mask_aug, 3, axis=-1).reshape(512, 512, 3)

    # 512 x 512 x 3
    return image_masked#.permute(1, 2, 0).numpy()

In [ ]:
def new_df_with_masks(df):

    new_df = {"image_masked": [],
              "nr_boil__nr_pan": []}

    for i, row in df.iterrows():
        #if i % 250 == 0:
        #    print(f"Processing image {i+1}/{len(df)}")

        # aprox. 5min

        all_polygons = row["polygons_pan"] + row["polygons_boil"]
        new_df["image_masked"].append(create_data(row["img_id"], all_polygons, data="train"))
        new_df["nr_boil__nr_pan"].append((sum(row["nr_boil"]), sum(row["nr_pan"])))

    return pd.DataFrame(new_df)

df_train_masks = new_df_with_masks(df_train)
df_val_masks = new_df_with_masks(df_val)

criados os dados, fazer um modelo:

- input: mask + rbg (512 x 512 x 3)

- output: 2 valores (boiler e pans)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

# Load the pre-trained model without the top classification layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(512, 512, 3))
base_model.trainable = True

for layer in base_model.layers[:80]:  
    layer.trainable = False


# Add new layers on top for your task
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(2, activation='linear')(x)

# Final model
forwardbone = Model(inputs=base_model.input, outputs=output)

#forwardbone.summary()  # Check the number of parameters


In [ ]:
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber


# Compile the model
forwardbone.compile(optimizer="AdamW",
                    loss=Huber(delta=1.0),
                    metrics=["mae"])

# Train the model
history = forwardbone.fit(
    np.array(df_train_masks['image_masked'].tolist()),
    np.array(df_train_masks['nr_boil__nr_pan'].tolist()),
    validation_data=(np.array(df_val_masks['image_masked'].tolist()),
                     np.array(df_val_masks['nr_boil__nr_pan'].tolist())),
    epochs=15,
    batch_size=32,
    callbacks=[callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)

In [ ]:
forwardbone.save("forwardbone.keras")

In [ ]:
# Evaluate on test set
test_loss, test_mae = forwardbone.evaluate(np.array(df_val_masks['image_masked'].tolist()),
                                           np.array(df_val_masks['nr_boil__nr_pan'].tolist()))
print("Test MAE:", test_mae)

# Predict counts on new images
predictions = forwardbone.predict(np.array(df_val_masks['image_masked'].tolist()))

# Display first 5 predictions vs actual values
for i in range(5):
    print(f"Predicted: {predictions[i]}, Actual: {np.array(df_val_masks['nr_boil__nr_pan'].tolist())[i]}")


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.yscale('log')
plt.show()

plt.plot(history.history['mae'], label='train mae')
plt.plot(history.history['val_mae'], label='val mae')
plt.legend()
plt.yscale('log')
plt.show()